In [1]:
import asyncio
import ipywidgets as widgets
from IPython.display import display
from intelino.trainlib_async import TrainScanner, Train

In [2]:
async def print_info_card(train: Train, out, full: bool = True):
    out.append_stdout(f'{"="*40}\n')
    out.append_stdout(f'{"Train info":^40s}\n')
    out.append_stdout(f'{"="*40}\n')
    out.append_stdout(f'{"Advertised name":<15s}{train.name:>25s}\n')
    out.append_stdout(f'{"ID":<4s}{train.id:>36s}\n')
    out.append_stdout(f'{"="*40}\n')

    if not full:
        return

    # any further information requires the train to be connected
    if not train.is_connected:
        await train.connect()

    msg = await train.get_mac_address()
    out.append_stdout(f'{"MAC address":<15s}{msg.mac_address:>25s}\n')

    msg = await train.get_uuid()
    out.append_stdout(f'{"UUID":<15s}{msg.uuid:>25s}\n')

    msg = await train.get_version_info()
    out.append_stdout(f'{"FW version":<15s}{str(msg.fw_version):>25s}\n')
    out.append_stdout(f'{"BLE API version":<15s}{str(msg.ble_api_version):>25s}\n')

    msg = await train.get_stats_lifetime_odometer()
    out.append_stdout(f'{"Odometer":<15s}{msg.lifetime_odometer_meters:18.2f} meters\n')

    out.append_stdout(f'{"="*40}\n')

In [3]:
async def train_info(out):
    # collection of our trains
    # find all trains (search for max 2 seconds)
    trains = await TrainScanner(timeout=2.0).get_trains(
        connect=False,
    )

    # inspect them one by one
    for train in trains:
        await print_info_card(train, out, full=True)
        await train.disconnect()
        out.append_stdout("Disconnected.")
        
    if len(trains) == 0:
        out.append_stdout("No trains found.")

In [4]:
button = widgets.Button(description="Scan for trains")
out = widgets.Output()

In [5]:
loop = asyncio.get_running_loop()

In [6]:
# This is how we run the task in Jupyter since there is already an event loop.
  
def on_button_clicked(b):
    out.clear_output()
    loop.create_task(train_info(out))

button.on_click(on_button_clicked)
display(button, out)

Button(description='Scan for trains', style=ButtonStyle())

Output()